<a href="https://colab.research.google.com/github/sharpened-ferret/CSC345-Object-Recognition/blob/main/Object_Recognition_Manual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVM Implementation of Object Recognition on a Subset of the CIFAR-100 Dataset

For this implementation, we are using a Support Vector Machine to classify the images in the dataset. 

This model is trained using Histograms of Oriented Gradients that are generated from the given data files. 

In [1]:
import numpy as np
import skimage.feature

First, we need to load the training data files. 
Here, BASE_PATH points to the folder containing the data files. 

In [2]:
BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/cw_data/'

images = np.load(BASE_PATH + '/trnImage.npy')
label_fine = np.load(BASE_PATH + '/trnLabel_fine.npy')
label_coarse = np.load(BASE_PATH + '/trnLabel_coarse.npy')

This cell contains a configurable limiter (currently disabled), for training the SVM using less than the entire dataset, to speed up incremental testing.

In [12]:
LIMITER = 10000

# images = images[:,:,:,:LIMITER]
# label_coarse = label_coarse[:LIMITER]
# label_fine = label_fine[:LIMITER]

To use the provided data with an SVM, it needs some initial processing to generate a Histogram of Oriented Gradients for each image in the data set.

This function takes a dataset, in the format provided for the coursework, and returns an array of Histograms, one for each image provided.

In [4]:
def gen_hogs(dataset):
  ppc = 4
  hog_features = []
  for image_num in range(dataset.shape[3]):
    image = dataset[:, :, :, image_num]
    hog_features.append(skimage.feature.hog(image, pixels_per_cell=(6,6), cells_per_block=(2,2), visualize=False))
    if image_num % 1000 == 0:
      # Prints the current image number every 1000 images to monitor progress
      print(image_num)
  return hog_features

Now that our helper function is defined, we can pass the training data into it to generate histograms for modeling. 

In [5]:
features = gen_hogs(images)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000


We take the histogram array generated, and convert it to an numpy array for easier processing. 

In [6]:
features_array = np.array(features)
print(features_array.shape)

(50000, 576)


Now we can train an SVM on the HOGs from the training data, first using the coarse label set. 

In [7]:
from sklearn import svm

svm = svm.SVC()

x_train = features_array
y_train = label_coarse

svm.fit(x_train, y_train)

SVC()

For evaluating the model, we first need to load the test data and generate HOGs for it using the same helper function defined earlier. 

In [8]:
test_images = np.load(BASE_PATH + '/tstImage.npy')
test_label_fine = np.load(BASE_PATH + '/tstLabel_fine.npy')
test_label_coarse = np.load(BASE_PATH + '/tstLabel_coarse.npy')

In [9]:
x_test = np.array(gen_hogs(test_images))
y_test = test_label_coarse

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


Then, we can run predictions on the model using the prepared test data, and evaluate the accuracy of the classification. 

In [10]:
y_predict = svm.predict(x_test)

In [11]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy: "+str(accuracy_score(y_test, y_predict)))
print(classification_report(y_test, y_predict))

Accuracy: 0.3939
              precision    recall  f1-score   support

           0       0.28      0.30      0.29       500
           1       0.36      0.38      0.37       500
           2       0.31      0.35      0.33       500
           3       0.57      0.59      0.58       500
           4       0.47      0.39      0.43       500
           5       0.56      0.43      0.49       500
           6       0.57      0.64      0.60       500
           7       0.33      0.37      0.35       500
           8       0.26      0.35      0.30       500
           9       0.45      0.53      0.48       500
          10       0.59      0.50      0.54       500
          11       0.28      0.29      0.28       500
          12       0.22      0.20      0.21       500
          13       0.30      0.22      0.25       500
          14       0.41      0.38      0.40       500
          15       0.25      0.24      0.24       500
          16       0.26      0.22      0.24       500
          